## Pikabu parser

In [12]:
import requests
import time
import re
from bs4 import BeautifulSoup
from pprint import pprint
from nltk import sent_tokenize

In [13]:
def get_posts(page: str, tag: str):
    soup = BeautifulSoup(page, "html.parser")
    stories = soup.find_all('div', {'class': 'story__main'})
    if stories == []:
        print(f'Nothing found for tag {tag}')
    else:
        with open('pikabu_sep_by_sent.csv', 'a', encoding='UTF-8') as f:
            for story in stories[:-1]: # because last post is an ad
                title = story.find('a', {'class': 'story__title-link'}).text.replace(', ', ',\t\n')
                f.write('\t\n'.join(sent_tokenize(title)))
                f.write('\t\n')
                post = story.find('div', {'class': 'story__content-inner'}).text.replace('\n\n', '')
                post = re.sub(regexp, '. \g<1>', post)
                f.write('\t\n'.join(sent_tokenize(post)))
                f.write('\t\n\n') # empty line between posts

In [17]:
def parse_section(tag: str):
    for n in range(1, 50): # 50 pages
        url = f'https://pikabu.ru/tag/{tag}?st=3&page={n}'
        headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
            }
        response = requests.get(url, headers=headers)
        page = response.text
        get_posts(page, tag)
    print(f'Тег {tag} обкачен') # Tag {tag} has been parsed

In [19]:
regexp = re.compile(' ?\.([А-ЯЁ])')
regexp2 = re.compile('(, |\. )')
tags = ['Ирония','Юмор', 'Сарказм']
for tag in tags:
    with open('pikabu_sep_by_sent.csv', 'a', encoding='UTF-8') as f:
        f.write(f'\n\n{tag.upper()}\n\n')
    parse_section(tag)

Тег Ирония обкачен
Тег Юмор обкачен
Тег Сарказм обкачен
